# Neo4j Config

In [1]:
NEO4J_URI="neo4j+s://0438cb2c.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="dD3l5MoTW-_pX6STAQamH73KnfewGbkoKBRgC_vO4c0"


In [2]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

#  Groq Api for Using LLM 

In [3]:
groq_api_key="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"

# LLM config 

In [4]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D81440ABC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D814444E50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

# pdf Loader and Recursice Text Splitter 

In [6]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔹 Ask user for PDF path
pdf_path = input("Enter the full path to your PDF file: ").strip()

# 🔹 Extract book title from filename (without .pdf extension)
book_title = os.path.splitext(os.path.basename(pdf_path))[0]

def load_and_split_pdf(path, book_title):
    loader = PyPDFLoader(path)
    docs = loader.load()

    for doc in docs:
        doc.metadata['book'] = book_title

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    return chunks

# 🔹 Load and chunk the user-selected book
chunks = load_and_split_pdf(pdf_path, book_title)

# ✅ Preview
print(f"\nLoaded {len(chunks)} chunks from '{book_title}'")
print("Example chunk:\n", chunks[0].page_content[:300])



Loaded 306 chunks from '1. The Metamorphosis, Franz Kafka'
Example chunk:
 THE 
METAMORPHOSIS
Franz kafka 
InfoBooks.org


In [7]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 🔹 Ask user for PDF path
pdf_path = input("Enter the full path to your PDF file: ").strip()

# 🔹 Extract book title from filename (without .pdf extension)
book_title = os.path.splitext(os.path.basename(pdf_path))[0]

def load_and_split_pdf(path, book_title):
    loader = PyPDFLoader(path)
    docs = loader.load()

    for doc in docs:
        doc.metadata['book'] = book_title

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    return chunks

# 🔹 Load and chunk the user-selected book
all_chunks = load_and_split_pdf(pdf_path, book_title)

# 🔹 Use only the first 20 chunks
chunks = all_chunks[:20]

# ✅ Preview
print(f"\nLoaded {len(chunks)} chunks from '{book_title}' (first 20 only)")
print("Example chunk:\n", chunks[0].page_content[:300])



Loaded 20 chunks from '1. The Metamorphosis, Franz Kafka' (first 20 only)
Example chunk:
 THE 
METAMORPHOSIS
Franz kafka 
InfoBooks.org


# LLMGraph Transformer for converting data into Graphs 

In [8]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

# Converting Chunks into Graphs 

In [9]:
graph_documents=llm_transformer.convert_to_graph_documents(chunks)

# defining Graph 

In [10]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

C:\Users\arqamahmed.ansari\AppData\Local\Temp\ipykernel_11520\3558848663.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


# Adding Knowledge graph into Ne04j

In [11]:
# Step 2: Upload graph
graph.add_graph_documents(graph_documents)

In [12]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Book {id: STRING}
Person {id: STRING}
Organization {id: STRING}
Workofliterature {id: STRING}
Concept {id: STRING}
Author {id: STRING}
Profession {id: STRING}
Thing {id: STRING}
Pain {id: STRING}
Action {id: STRING}
Feeling {id: STRING}
Furniture {id: STRING}
Body_part {id: STRING}
Mark {id: STRING}
Object {id: STRING}
Event {id: STRING}
Relationship properties:

The relationships:
(:Book)-[:AUTHOR]->(:Person)
(:Book)-[:PUBLISHER]->(:Organization)
(:Book)-[:WROTE]->(:Author)
(:Person)-[:AUTHOR]->(:Workofliterature)
(:Person)-[:WROTE]->(:Book)
(:Person)-[:TRANSFORMSINTO]->(:Concept)
(:Person)-[:DREAMS]->(:Concept)
(:Person)-[:TRANSFORMED]->(:Concept)
(:Person)-[:OCCUPATION]->(:Profession)
(:Person)-[:LOOKING_AT]->(:Thing)
(:Person)-[:EMPLOYEE]->(:Person)
(:Person)-[:SPOKETO]->(:Person)
(:Person)-[:NEAR]->(:Object)
(:Person)-[:NEAR]->(:Furniture)
(:Person)-[:EXPERIENCE]->(:Pain)
(:Person)-[:PERFORMS]->(:Action)
(:Person)-[:EXPERIENCES]->(:Action)
(:Person)-[:EXPERIENCES]

# Defining GraphCypherQAChain to Query with Graph data in neo4j

In [13]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True  # ✅ This unlocks full Cypher access
)





In [ ]:
response=chain.invoke({"query":"YNOPSIS OF THE METAMORPHOSI?"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:Book)-[:AUTHOR]->(p:Person)
RETURN p.id, b.id 

Full Context:
[{'p.id': 'Franz Kafka', 'b.id': 'The Metamorphosis'}]

> Finished chain.


{'query': 'What is the main theme of the book?',
 'result': "I don't know the answer. \n"}